<a href="https://colab.research.google.com/github/hate-speech-classification/abusive-level-classification/blob/main/abusive_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
"""
Init
"""
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
"""
Read data from csv (url from github)
"""
url_train_data = 'https://raw.githubusercontent.com/amandacurry/convabuse/main/2_splits/ConvAbuseEMNLPtrain.csv'
df = pd.read_csv(url_train_data)

"""
Create new column labels
"""
def categorise(row):
  if row['Annotator1_is_abuse.1'] == '1':
    return 'Not abusive'
  elif row['Annotator1_is_abuse.0'] == '1':
    return "Ambiguous"
  elif row['Annotator1_is_abuse.-1'] == '1':
    return "Mildly abusive"
  elif row['Annotator1_is_abuse.-2'] == '1':
    return "Strongly abusive"
  elif row['Annotator1_is_abuse.-3'] == '1':
    return "Very strongly abusive"

my_tags = ['Not abusive', 'Ambiguous', 'Midly abusive', 'Strongly abusive', 'Very strongly abusive']
df['abusive_level'] = df.apply(lambda row: categorise(row), axis=1) 
df = df[pd.notnull(df['abusive_level'])]
# df.tail(10)

In [47]:
"""
Clean text (user column)
"""
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['user'] = df['user'].apply(clean_text)
data = df[['user','abusive_level']]
X = data.user
Y = data.abusive_level
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)
# data.tail(10)

In [55]:
"""
Naive Bayes Classifier 
Idea: text is categorized based on TF*IDF (term frequency * inverse data frequency)
"""

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

# %%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.786096256684492
                       precision    recall  f1-score   support

          Not abusive       0.00      0.00      0.00         6
            Ambiguous       0.00      0.00      0.00        16
        Midly abusive       0.79      1.00      0.88       144
     Strongly abusive       0.60      0.20      0.30        15
Very strongly abusive       0.00      0.00      0.00         6

             accuracy                           0.79       187
            macro avg       0.28      0.24      0.24       187
         weighted avg       0.66      0.79      0.70       187



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
"""
Linear Support Vector Machine
Idea: text is categorized based on TF*IDF (term frequency * inverse data frequency)
"""

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

# %%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.8021390374331551
                       precision    recall  f1-score   support

          Not abusive       0.00      0.00      0.00         6
            Ambiguous       0.33      0.12      0.18        16
        Midly abusive       0.85      0.97      0.90       144
     Strongly abusive       0.57      0.53      0.55        15
Very strongly abusive       0.50      0.17      0.25         6

             accuracy                           0.80       187
            macro avg       0.45      0.36      0.38       187
         weighted avg       0.74      0.80      0.76       187



In [57]:
"""
Logistic Regression
Idea: text is categorized based on TF*IDF (term frequency * inverse data frequency)
"""


from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

# %%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.7593582887700535
                       precision    recall  f1-score   support

          Not abusive       0.00      0.00      0.00         6
            Ambiguous       0.19      0.19      0.19        16
        Midly abusive       0.86      0.93      0.89       144
     Strongly abusive       0.50      0.27      0.35        15
Very strongly abusive       0.33      0.17      0.22         6

             accuracy                           0.76       187
            macro avg       0.38      0.31      0.33       187
         weighted avg       0.73      0.76      0.74       187



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
